In [1]:
# all imports up here, rerun this block when adding
import pandas as pd
import numpy as np
import cv2 as cv

In [2]:
#assumes dataset in folder
wines   = pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_1K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
ratings = pd.read_csv("C:\\Users\\Dell\\reommenders\\RecommenderSystems\\XWines_Slim_150K_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
len(wines), len(ratings)

print(ratings)
print(wines)

        RatingID   UserID  WineID Vintage  Rating                 Date
0            143  1356810  103471    1950     4.5  2021-11-02 20:52:59
1            199  1173759  111415    1951     5.0  2015-08-20 17:46:26
2            348  1164877  111395    1952     5.0  2020-11-13 05:40:26
3            374  1207665  111433    1953     5.0  2017-05-05 06:44:13
4            834  1075841  111431    1955     5.0  2016-09-14 20:18:38
...          ...      ...     ...     ...     ...                  ...
149995  21013438  1000052  111468    N.V.     4.5  2021-12-22 21:03:51
149996  21013467  1180844  111461    N.V.     4.0  2017-04-23 21:07:55
149997  21013494  1218581  113690    N.V.     3.5  2019-04-14 17:45:08
149998  21013505  1106198  111468    N.V.     4.5  2021-07-10 07:00:15
149999  21013509  1059173  111479    N.V.     5.0  2019-09-08 15:42:28

[150000 rows x 6 columns]
      WineID                                    WineName       Type  \
0     100001                          Espumante Mo

In [3]:
def read():
    ##assumes dataset in folder
    wines   = pd.read_csv("XWines_Full_100K_wines.csv", low_memory=False, encoding="utf-8", memory_map=True)
    ratings = pd.read_csv("XWines_Full_21M_ratings.csv", low_memory=False, encoding="utf-8", memory_map=True)
    return wines,ratings

In [4]:
# Step 2: Get user input
wine_type = input("Enter the type of wine (e.g., red, white, rosé, sparkling): ").lower()
wine_country = input("Enter the country of the wine (e.g., Brazil, New Zealand): ").lower()
wine_body = input("Enter the body of the wine(e.g. ligth-bodied, medium-bodied, full-bodied): ").lower()

# Step 3: Check if there are matches for both preferences
filtered_wines = wines.copy()

filtered_wines = wines[(wines['Type'].str.lower() == wine_type) 
                       & (wines['Country'].str.lower() == wine_country)
                       & (wines['Body'].str.lower() == wine_body)]

if filtered_wines.empty:
    filtered_wines = wines[(wines['Type'].str.lower() == wine_type)
                           & (wines['Country'].str.lower() == wine_country)]
    if not filtered_wines.empty:
        # Step 5: Notify the user that the region was left out
        print(f"No '{wine_type}' wines in country '{wine_country}' found with body '{wine_body}'")
        print(f"Showing all '{wine_type}' wines from country '{wine_country}' instead.")
    else:
        filtered_wines = wines[wines['Type'].str.lower() == wine_type]
        if not filtered_wines.empty:
            print(f"No '{wine_type}' wines in country '{wine_country}'. Showing all '{wine_type}'instead")
        else:
            # No wines matching even the type
            print(f"No wines found for the type '{wine_type}'.")
else:
    # Found wines for both type and region
    print(f"Showing '{wine_type}' wines from '{wine_country}'.")

# Step 6: Display the result
if not filtered_wines.empty:
    print(filtered_wines)
else:
    print("No wines found that match your preferences.")

No 'red' wines in country 'france' found with body 's'
Showing all 'red' wines from country 'france' instead.
     WineID                             WineName Type  \
187  111395  Pauillac (Premier Grand Cru Classé)  Red   
188  111398                                Rouge  Red   
189  111415  Pauillac (Premier Grand Cru Classé)  Red   
190  111417          Pauillac (Grand Cru Classé)  Red   
191  111420                  Beaujolais-Villages  Red   
..      ...                                  ...  ...   
358  118457                    Fleurie La Madone  Red   
359  118927        Nuits-St-Georges Les Plateaux  Red   
361  120737            Les Dessous des Dentelles  Red   
364  125678  Les Fuées Chambolle-Musigny 1er Cru  Red   
371  132977      Bandol Cuvée Spéciale La Louffe  Red   

                         Elaborate  \
187               Assemblage/Blend   
188                  Varietal/100%   
189  Assemblage/Bordeaux Red Blend   
190               Assemblage/Blend   
191            

In [5]:
filtered_reviews = ratings[ratings['WineID'].isin(filtered_wines['WineID'])]

# Print the filtered reviews DataFrame
print(filtered_reviews)


        RatingID   UserID  WineID Vintage  Rating                 Date
1            199  1173759  111415    1951     5.0  2015-08-20 17:46:26
2            348  1164877  111395    1952     5.0  2020-11-13 05:40:26
3            374  1207665  111433    1953     5.0  2017-05-05 06:44:13
4            834  1075841  111431    1955     5.0  2016-09-14 20:18:38
5            876  1211463  111395    1955     5.0  2021-12-02 23:12:49
...          ...      ...     ...     ...     ...                  ...
140300  20566965  1291467  111420    2021     3.5  2021-11-24 11:32:24
140316  20568303  1228593  111420    2021     3.0  2021-11-24 02:32:31
140325  20569124  1060915  111420    2021     4.0  2021-12-12 06:39:52
140334  20569465  1136971  111420    2021     3.5  2021-12-14 17:36:43
140335  20569523  1215842  111420    2021     3.0  2021-11-23 17:22:38

[22968 rows x 6 columns]


In [6]:
from IRS import contentBased

contentBased_class = contentBased(filtered_reviews, filtered_wines)


print(contentBased_class.recommend_wines(1173759, 5, False))


187    ['Cabernet Sauvignon', 'Cabernet Franc', 'Merl...
188                                         ['Grenache']
189                     ['Merlot', 'Cabernet Sauvignon']
190    ['Cabernet Sauvignon', 'Merlot', 'Cabernet Fra...
191                                       ['Gamay Noir']
                             ...                        
358                                       ['Gamay Noir']
359                                       ['Pinot Noir']
361            ['Grenache', 'Mourvedre', 'Syrah/Shiraz']
364                                       ['Pinot Noir']
371                ['Mourvedre', 'Grenache', 'Cinsault']
Name: Grapes, Length: 92, dtype: object
[111433, 111395, 113193, 112353, 111429]


c:\Users\Dell\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Dell\reommenders\RecommenderSystems\Individual_Recommenders\IRS.py:53: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  wine_data['Grapes'] = wine_data['Grapes'].apply(lambda x: ' '.join(eval(x)))  # Convert list to space-separated string
c:\Users\Dell\reommenders\RecommenderSystems\Individual_Recommenders\IRS.py:54: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See t